In [14]:
import numpy as np
from tqdm import tqdm
from time import time
import json, pickle, os, string, tqdm, kenlm, json
from collections import defaultdict, Counter
from itertools import groupby
import Levenshtein as Lev
import math


In [15]:
def lse(*args): # logsumexp trick
    a_max = max(args)
    lsp = math.log(sum(math.exp(a-a_max) for a in args))
    return a_max + lsp

In [16]:
#s1 = True text
#s2 = predicted text

def wer_(s1, s2):
    """
    Computes the Word Error Rate, defined as the edit distance between the
    two provided sentences after tokenizing to words.
    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """

    # build mapping of words to integers
    b = set(s1.split() + s2.split())
    word2char = dict(zip(b, range(len(b))))

    # map the words to a char array (Levenshtein packages only accepts
    # strings)
    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]
    
    return Lev.distance(''.join(w1), ''.join(w2))

def cer_(s1, s2):
    """
    Computes the Character Error Rate, defined as the edit distance.

    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """
    s1, s2, = s1.replace(' ', ''), s2.replace(' ', '')

    return Lev.distance(s1, s2)




#When using the above implementation, use the code belove to calculate the wer in percentatge: 
#pred = list of ouput prediction of model (it is the text) # example [" MY NAME IS HEMANT", " I AM A GOD"]
# total_wer = 0
# for x in range(len(pred)):
#     transcript, reference = data_[x][1], pred[x]
#     wer_inst = wer(transcript, reference)
#     total_wer += float(wer_inst)
# print("WER is : ",total_wer/len(pred),"%")
# wer_(pred,true)/len(pred.split(' '))*100

In [17]:
out = np.load("out.npy")
with open("true.txt", "r") as f:
    reference = f.read()
with open("pred.txt", "r") as f:
    transcript = f.read()

In [18]:
def ctc_best_path(out,labels):
    "implements best path decoding as shown by Graves"
    out = [labels[i] for i in np.argmax(out, axis=1) if i!=labels[-1]]
    o = ""
    for i,j in groupby(out):
        o = o + i
    return o.replace("_","")

In [19]:
labels = "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ "

In [20]:
gred_txt = ctc_best_path(out,labels)
print(gred_txt)
wer_(gred_txt,reference)/len(reference.split(' '))*100

OUR PRODATIVITY AND OUR COMMITMENT TO OUR CLIENTS XXXUM TO MAKE SURE THAT WE NEED WHAT BE XXXUM PROMISED FOR THE DEADLINE


19.047619047619047

### WORD LM Implementation

In [54]:
lm_w = kenlm.LanguageModel('/home/hemant/sopi_deep/lm/3_gram_full.arpa')

In [55]:
def sort_beam(ptot,k):
    if len(ptot) < k:
        return [i for i in ptot.keys()]
    else:
        dict_ = sorted(dict((v,k) for k,v in ptot.items()).items(),reverse=True)[:k]
        return [i[1] for i in dict_]

#using WORD LM
def ctc_beam_search(out,labels, prune=0.0001, k=20, lm=None,alpha=0.3,beta=12):
    "implements CTC Prefix Search Decoding Algo13.043478260869565%'rithm as shown by Graves"
    '''
    out = ctc output
    labels = string of labels
    prune = prune the ctc output
    k=beam-width
    lm=word age model used
    alpha,beta = hyper-parameters
    '''

    bc_i = 0 # blank/special charatcter index 
    F = out.shape[1]
    out = np.vstack((np.zeros(F), out))
    steps = out.shape[0]
    
    pb, pnb = defaultdict(Counter), defaultdict(Counter)
    pb[0][''], pnb[0][''] = 1, 0
    prev_beams = ['']
    for t in range(1,steps):
        pruned_alphabet = [labels[i] for i in np.where(out[t] > prune)[0]]
        for b in prev_beams:
            
            for c_t in pruned_alphabet:
                index = labels.index(c_t)
                #Collapsing case (copy case as the last character in the beam)
                if c_t == "_": #Extending with a blank
                    pb[t][b] = out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    continue
                else:
                    i_plus = b + c_t
                    if len(b) > 0 and c_t == b[-1]: #Extending with the same character as the last one
                        pnb[t][b] += out[t][index]*pnb[t-1][b]
                        pnb[t][i_plus] += out[t][index]*pb[t-1][b]
                    #Extending case as the last character is different
                    elif c_t == " " and len(b.replace(' ', '')) > 0 : # LM constraints
                        prob = [i[0] for i in lm.full_scores(i_plus,eos=False,bos=False)][-1]
                        lm_p = (10**prob)**alpha
                        pnb[t][i_plus] += lm_p*out[t][index]*(pb[t-1][b] + pnb[t-1][b])*(len(b.split())+1)**beta
                    else:
                        pnb[t][i_plus] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    # If the new beam is not in the previous beams
                    if i_plus not in prev_beams:
                        pb[t][i_plus] += out[t][labels.index("_")] * (pb[t - 1][i_plus] + pnb[t - 1][i_plus])
                        pnb[t][i_plus] += out[t][index] * pnb[t - 1][i_plus]

        ptot = pb[t] + pnb[t]
        prev_beams = sort_beam(ptot,k)
    return prev_beams[0]

In [78]:
beam_txt = ctc_beam_search(out,labels,0.0001,k=100,lm=lm_w,alpha=1.2,beta=4)
print(beam_txt)
wer_(beam_txt,reference)/len(reference.split(' '))*100

OUR PRODUCTIVITY AND OUR COMMITMENT TO OUR CLIENTS XXXUM TO MAKE SURE THAT WE NEED WHAT WE XXXUM PROMISED FOR THE DEADLINE


9.523809523809524

In [337]:
import numpy as np
import math
import collections

NEG_INF = -float("inf")

def make_new_beam():
  fn = lambda : (NEG_INF, NEG_INF)
  return collections.defaultdict(fn)

def logsumexp(*args):
  """
  Stable log sum exp.
  """
  if all(a == NEG_INF for a in args):
      return NEG_INF
  a_max = max(args)
  lsp = math.log(sum(math.exp(a - a_max)
                      for a in args))
  return a_max + lsp


In [640]:
def length(sequence):
    for i in h.items():
        m = i[0]
    g = ''
    for i in m:
        g+=str(i)

def decode(probs, beam_size=2, blank=0,beta=0):
    T, S = probs.shape
    probs = np.log(probs)
    beam = [(tuple(), (0.0, NEG_INF))]

    for t in range(T): # Loop over time
        next_beam = make_new_beam()

        for prefix, (p_b, p_nb) in beam: # Loop over beam
            
            for s in range(S): # Loop over vocab
                p = probs[t, s]
                if s == blank:
                    n_p_b, n_p_nb = next_beam[prefix]
                    n_p_b = logsumexp(n_p_b, p_b + p, p_nb + p)
                    next_beam[prefix] = (n_p_b, n_p_nb)
                    continue
                
                end_t = prefix[-1] if prefix else None
                n_prefix = prefix + (s,)
                n_p_b, n_p_nb = next_beam[n_prefix]
                
                if s != end_t:
                    n_p_nb = logsumexp(n_p_nb, p_b + p, p_nb + p)
                else:
                    n_p_nb = logsumexp(n_p_nb + p, p_b + p)
                
                next_beam[n_prefix] = (n_p_b, n_p_nb)
                if s == end_t:
                    n_p_b, n_p_nb = next_beam[prefix]
                    n_p_nb = logsumexp(n_p_nb + p, p_nb + p)
                    next_beam[prefix] = (n_p_b, n_p_nb)
        beam = sorted(next_beam.items(),
                key=lambda x : (logsumexp(*x[1])),
                reverse=True)
        beam = beam[:beam_size]

    best = beam[0]
    return best[0], -logsumexp(*best[1]),next_beam


In [641]:
label, score, h = decode(out)
print("Score {:.3f}".format(score))
a =''
for i in label:
    a += labels[i]
print("\n",a)
wer_(a,reference)/len(reference.split(' '))*100

Score 9.169

 OUR PRODATIVITY AND OUR COMMITMENT TO OUR CLIENTS XXXUM TO MAKE SURE THAT WE NEED WHAT BE XXXUM PROMISED FOR THE DEADLINE


19.047619047619047

In [642]:
beam = sorted(next_beam.items(),
                key=lambda x : logsumexp(*x[1]),
                reverse=True)